In [7]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt 

In [8]:
PATH = "../kaggle/data/"

In [9]:
df_0 = pd.read_csv(PATH + "train-metadata.csv")

/tmp/ipykernel_48515/1877868789.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_0 = pd.read_csv(PATH + "train-metadata.csv")


In [10]:
# Clean and preprocess data for training and testing 

cols_to_drop = ["isic_id", "patient_id", "mel_thick_mm", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "attribution", "copyright_license", "tbp_lv_location", "tbp_lv_location_simple", "tbp_lv_dnn_lesion_confidence", "anatom_site_general", "image_type", "tbp_tile_type"]
df_0.drop(columns=cols_to_drop, inplace=True)
df_0["sex"] = df_0["sex"].replace({"male": 1, "female": 0})

# replace all NaNs with np.nan
df_0.replace("", np.nan, inplace=True)

# Drop any column with a string vallue 
for col in df_0.columns:
    if df_0[col].dtype == "object":
        df_0.drop(columns=[col], inplace=True)

# clean up for training, split into train and test
train_frac = 0.8 
train_size = int(train_frac * len(df_0))

train = df_0[:train_size]
x_train = df_0.drop(columns=["target"])
y_train = df_0["target"]

test = df_0[train_size:]
x_test = df_0.drop(columns=["target"])
y_test = df_0["target"]

In [11]:
# Define the NN 
inputs = keras.Input(shape=(len(x_train.keys()),)) # One input for each feature in the dataset 

# Add two hidden layers with 64 units each and ReLU activation
x = layers.Dense(64, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)

# Define the output layer with a single unit and sigmoid activation
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="isic-v0")


In [12]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history = model.fit(x_train, y_train, batch_size=512, epochs=3, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/3


/home/ubuntu/miniconda3/envs/tf-v2/lib/python3.8/site-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2477/2507 [============================>.] - ETA: 0s - loss: 0.0466 - binary_accuracy: 0.9989

/home/ubuntu/miniconda3/envs/tf-v2/lib/python3.8/site-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2507/2507 [==============================] - 4s 1ms/step - loss: 0.0461 - binary_accuracy: 0.9989 - val_loss: 0.0066 - val_binary_accuracy: 0.9992
Epoch 2/3
2507/2507 [==============================] - 3s 1ms/step - loss: 0.0082 - binary_accuracy: 0.9990 - val_loss: 0.0066 - val_binary_accuracy: 0.9992
Epoch 3/3
2507/2507 [==============================] - 4s 1ms/step - loss: 0.0082 - binary_accuracy: 0.9990 - val_loss: 0.0066 - val_binary_accuracy: 0.9992
12534/12534 - 9s - loss: 0.0078 - binary_accuracy: 0.9990 - 9s/epoch - 683us/step
Test loss: 0.007798830047249794
Test accuracy: 0.9990200996398926


In [13]:
test_df = pd.read_csv(PATH + "test-metadata.csv")
isic_ids = test_df["isic_id"]

try: 
    test_df.drop(columns=cols_to_drop, inplace=True)
except KeyError: 
    pass

test_df.replace("", np.nan, inplace=True)
test_df["sex"] = test_df["sex"].replace({"male": 1, "female": 0})

# Drop any column with a string vallue 
for col in test_df.columns:
    if test_df[col].dtype == "object":
        test_df.drop(columns=[col], inplace=True)

In [14]:
preds = model.predict(test_df)

df_sub = pd.DataFrame({"isic_id": isic_ids, "target": preds[:,0]})
df_sub.to_csv("submission.csv", index=False)


1/1 [==============================] - 0s 66ms/step
